# Model Selection and Performance Testing<br>
Results from the model_selection_v0 python notebook got me here, which is testing some optimized Logistic Regression models for use in predicting risk of 7 day mortality. In this notebook, I'll be finalizing the models and reporting on performance of the trained models on validation data, using 10-fold cross validation. <br><br>

Additionally, other models will be ran (kNN, decision tree, maybe neural net) to show other future options, show a performance ceiling given the data inputs, despite the lack of model interpretability. I think a long-term modeling option is deep learning methods, or even XGboost, but right now, the models need to be fully interpretable. Thus, the focus on logistic regression.<br><br>

While the previous notebook identified median columnar imputation of missing data as the superior method (when compared to adding 0 for missing data), I'm still going to test each model version against both methods of imputation, as I think it would be worthwhile to quantify the improvements in model performance with median imputation. Additionally, seeing errors for each imputation method will be informative. <br><br>

One data processing step I would like to also test is z-scaling (normalizing data to mean = 0, sd = 1) for all data. By doing this, I think I will get a better representation of which variables appear to be most informative of the outcome, which again will help with interpretability clinically. While yes, the scales themselves will need to be re-converted for direct interpretation, I don't think this will be necessary, as most people right now just want to know WHAT is informative, not necessarily how much.

In [1]:
%matplotlib inline

#load libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeClassifier 

from IPython.display import display #displays full dataframe columns
#display all dataframe columns when printed
pd.options.display.max_columns = None

C:\tooling\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\tooling\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#load data
df = pd.read_csv('C:/Users/Mark.Burghart/Documents/projects/hospice_carepoint/data/transformed/carepoint_transformed_dummied.csv', index_col=0)
df.shape

### Train/Test Split<br>
Same split and randomizer will be used from the previous model_selection_v0 notebook

In [ ]:
#separate variables (X) from outcome of interest (y)
df.shape
cols = df.columns.get_values() #converts column names to list
cols = cols.tolist()
feature_cols = [x for x in cols if x != 'death_within_7_days'] #removes outcome of interest from list ('death_within_7_days')

#extract rows
#print(feature_cols) #debug
X = df.loc[:, feature_cols]
X.shape #outcome column has been removed

In [ ]:
#save outcome variable as y
y = df.death_within_7_days
y.shape

In [ ]:
#separate data into training/test (aka holdout) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 23) #random_state for reproducibility

In [ ]:
#create objects with specific imputation method
#impute missing values: replacing NaNs with Median Column value for each column
X_train_med = X_train.fillna(X_train.median()) 
y_train_med = y_train.fillna(y_train.median())

#impute missing values: replacing NaNs with 0
X_train_zero = X_train.fillna(0)
y_train_zero = y_train.fillna(0)